# GLOBE data sources

In [ ]:
# package setup

import requests
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import sys
import utm
import rasterio
import rasterio.features

In [ ]:
# download mosquito data from GLOBE API and write to csv

base_url = 'https://api.globe.gov/search/v1/measurement/protocol/'

query_params = {
  'protocols': 'mosquito_habitat_mapper',
  'geojson': "TRUE",
  'sample': "FALSE"
}

mosquito_data_json = requests.get(base_url, params = query_params).json()

parsedData = []

for obs in mosquito_data_json['features']:
    parsedObs = obs['properties']

    latitude = obs['geometry']['coordinates'][1]
    longitude = obs['geometry']['coordinates'][0]

    parsedObs['latitude'] = latitude
    parsedObs['longitude'] = longitude

    parsedData.append(parsedObs)

df = pd.DataFrame(parsedData).drop(
    columns=[
        "mosquitohabitatmapperMeasurementLongitude", 
        "mosquitohabitatmapperMeasurementLatitude"
    ]
)
gdf = gpd.GeoDataFrame(
    df, 
    geometry = gpd.points_from_xy(df.longitude, df.latitude),
    crs = {'init' :'epsg:4326'}
)
df.to_csv('processed/mosquito-observations.csv')

In [ ]:
# here's what the data looks like...
pruned_gdf = gdf.rename(columns = {
    "countryName": "country",
    "mosquitohabitatmapperMeasuredAt": "timestamp",
    "mosquitohabitatmapperWaterSource": "waterSource",
    "mosquitohabitatmapperWaterSourceType": "sourceType",
    "mosquitohabitatmapperLarvaeCount": "larvaeCount",
    "mosquitohabitatmapperMosquitoAdults": "adults",
    "mosquitohabitatmapperMosquitoEggs": "eggs",
    "mosquitohabitatmapperMosquitoPupae": "pupae",
    "mosquitohabitatmapperGenus": "genus",
    "mosquitohabitatmapperSpecies": "species",
    
}).drop(columns = {
    "countryCode",
    "mosquitohabitatmapperBreedingGroundEliminated",
    "elevation",
    "mosquitohabitatmapperDataSource",
    "mosquitohabitatmapperMeasurementElevation",
    "mosquitohabitatmapperMosquitoHabitatMapperId",
    "mosquitohabitatmapperUserid",
    "organizationId",
    "organizationName",
    "protocol",
    "siteId",
    "siteName",
    "mosquitohabitatmapperWaterSourcePhotoUrls",
    "mosquitohabitatmapperLastIdentifyStage",
    "mosquitohabitatmapperComments",
    "geometry",
    "mosquitohabitatmapperLarvaFullBodyPhotoUrls",
    "mosquitohabitatmapperAbdomenCloseupPhotoUrls"
})

pruned_gdf.tail(20)

## Raster map of world

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(figsize = (25, 15))
gdf.plot(ax = ax, color = "red", markersize = 3)

In [ ]:
raster = rasterio.features.rasterize(
  [(x.geometry, 1) for i, x in world.iterrows()],
    out_shape=(180, 360),
    transform=rasterio.Affine(1,0,-180,0,-1,90)
)

np.savetxt("processed/raster-world-map.txt", raster, delimiter = "", fmt = "%1u")

plt.imshow(raster)